In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [11]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories

In [14]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root]) # it is a box type

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [15]:
import os 
import urllib.request as request
import zipfile
from src.cnnClassifier import logger
from src.cnnClassifier.utils.common import get_size

In [ ]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download with following info: \n {headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [20]:
# pipleline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-07-04 21:22:17,868: INFO: common: YAML file loaded successfully: config/config.yaml]
[2025-07-04 21:22:17,872: INFO: common: YAML file loaded successfully: params.yaml]
[2025-07-04 21:22:17,899: INFO: common: Created Directory at: artifacts]
[2025-07-04 21:22:17,905: INFO: common: Created Directory at: artifacts/data_ingestion]
[2025-07-04 21:22:19,361: INFO: 3553833483: artifacts/data_ingestion/data.zip download with following info: 
 Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 8601:14F01A:2E2E45:3E5287:68689A68
Accept-Ranges: bytes
Date: Sat, 05 Jul 2025 03:22:19 GMT
Via: 1.1 varnish
X-Served-By: cache-den-kden1300030-DEN
X-

Traceback (most recent call last):
  File "/Users/uw-user/Library/CloudStorage/GoogleDrive-vvithurshan@gmail.com/My Drive/Artificial-Intelligence-2025/Machine-Learning/PW-Skills/Projects/Chiken-Disease-Classification/venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3699, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/0x/vxdg7v_92jn9xvqfpln1gdk80000gr/T/ipykernel_76383/3901236409.py", line 9, in <module>
    raise e
  File "/var/folders/0x/vxdg7v_92jn9xvqfpln1gdk80000gr/T/ipykernel_76383/3901236409.py", line 7, in <module>
    data_ingestion.extract_zip_file()
  File "/var/folders/0x/vxdg7v_92jn9xvqfpln1gdk80000gr/T/ipykernel_76383/3553833483.py", line 23, in extract_zip_file
    with zipfile.ZipExtFile(self.config.local_data_file, 'r') as zip_ref:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: ZipExtFile.__init__() missing 1 required positional argument: 'zipinfo'

During handling of the above ex